In [19]:
from keras.layers import Input, Dense
from keras.layers.core import Reshape
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers.convolutional import MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.convolutional import Convolution2D as Conv2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adadelta, RMSprop
import os
import os.path
import numpy as np
from PIL import Image
from numpy import * 
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
import scipy.misc
import cv2

In [20]:
input_img = Input(shape=(128,128,1))
enco = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
enco = BatchNormalization()(enco)
enco = Conv2D(16, (3, 3), activation='relu', padding='same')(enco)
enco = BatchNormalization()(enco)
enco = MaxPooling2D(pool_size=(2, 2))(enco)
   
enco = Conv2D(32, (3, 3), activation='relu', padding='same')(enco)
enco = BatchNormalization()(enco)
enco = Conv2D(32, (3, 3), activation='relu', padding='same')(enco)
enco = BatchNormalization()(enco)
enco = MaxPooling2D(pool_size=(2, 2))(enco)
      
enco = Conv2D(64, (3, 3), activation='relu', padding='same')(enco)
enco = BatchNormalization()(enco)
enco = Conv2D(64, (3, 3), activation='relu', padding='same')(enco)
enco = BatchNormalization()(enco)
enco = MaxPooling2D(pool_size=(2, 2))(enco)
    
enco = Conv2D(128, (3, 3), activation='relu', padding='same')(enco)
enco = BatchNormalization()(enco)
enco = Conv2D(128, (3, 3), activation='relu', padding='same')(enco)
enco = BatchNormalization()(enco)
enco = MaxPooling2D(pool_size=(2, 2))(enco)
    
enco = Conv2D(256, (3, 3), activation='relu', padding='same')(enco)
enco = BatchNormalization()(enco)
enco = Conv2D(256, (3, 3), activation='relu', padding='same')(enco)
enco = BatchNormalization()(enco)

In [21]:
deco = UpSampling2D((2,2))(enco)
deco = Conv2D(128, (3, 3), activation='relu', padding='same')(deco)
deco = BatchNormalization()(deco)
deco = Conv2D(128, (3, 3), activation='relu', padding='same')(deco)
deco = BatchNormalization()(deco)

deco = UpSampling2D((2,2))(deco)
deco = Conv2D(64, (3, 3), activation='relu', padding='same')(deco)
deco = BatchNormalization()(deco)
deco = Conv2D(64, (3, 3), activation='relu', padding='same')(deco)
deco = BatchNormalization()(deco)

deco = UpSampling2D((2,2))(deco)
deco = Conv2D(32, (3, 3), activation='relu', padding='same')(deco)
deco = BatchNormalization()(deco)
deco = Conv2D(32, (3, 3), activation='relu', padding='same')(deco)
deco = BatchNormalization()(deco)
   
deco = UpSampling2D((2,2))(deco)
deco = Conv2D(16, (3, 3), activation='relu', padding='same')(deco)
deco = BatchNormalization()(deco)
deco = Conv2D(16, (3, 3), activation='relu', padding='same')(deco)
deco = BatchNormalization()(deco)

decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(deco)
    

In [22]:
autoencoder = Model(input_img, decoded)

In [23]:
encoder = Model(input_img, enco)

In [24]:
#ada=Adadelta(lr=5.0, rho=0.95, epsilon=1e-08, decay=0.001)
rms=RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.001)
autoencoder.compile(loss='mean_squared_error', optimizer=rms)
encoder.compile(loss='mean_squared_error', optimizer=rms)

basic_mat=[]
tobe_mat=[]

In [25]:
path1="Data"

In [26]:
for i in range(1,124):
    path_major=path1+'/'+str(i)
    for j in range(1,101):
        img=array(Image.open(path_major+"/"+str(j)+"_.jpg"))
        #print shape(img)
        img = cv2.cvtColor( img, cv2.COLOR_RGB2GRAY )
        img=img.reshape(128,128,1)
        basic_mat.append(img)
        tobe_mat.append(img)

In [27]:
data,Label = shuffle(basic_mat,tobe_mat, random_state=2)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(data, Label, test_size=0.2, random_state=2)
X_train = array(X_train)
y_train = array(y_train)
X_test = array(X_test)
y_test = array(y_test)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(9840, 128, 128, 1)
(9840, 128, 128, 1)
(2460, 128, 128, 1)
(2460, 128, 128, 1)


In [29]:
x_train = X_train.astype('float32') / 255.
x_test = X_test.astype('float32') / 255.

y_train = y_train.astype('float32') / 255.
y_test = y_test.astype('float32') / 255.
CheckDir = 'sample/'

In [ ]:
for epoch in range(1,300):
    
    train_X,train_Y=shuffle(x_train,y_train)
    print ("Epoch is: %d\n" % epoch)
    batch_size=64
    print ("Number of batches: %d\n" % int(len(train_X)/batch_size))
    num_batches=int(len(train_X)/batch_size)
    for batch in range(num_batches):    
        batch_train_X=train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
        batch_train_Y=train_Y[batch*batch_size:min((batch+1)*batch_size,len(train_Y))]
        loss=autoencoder.train_on_batch(batch_train_X,batch_train_Y)
        print ('epoch_num: %d batch_num: %d loss: %f\n' % (epoch,batch,loss))

    autoencoder.save_weights("fabric_autoen_new.h5")
    encoder.save_weights("Only_Enco.h5")
    if(epoch%2==0):
        x_test,y_test=shuffle(x_test,y_test)
        decoded_imgs=autoencoder.predict(x_test[:2])
        temp = np.zeros([128, 128*3,3])
        temp[:, :128,:1] = x_test[0,:,:,:1]
        temp[:, 128:128*2,:1] = y_test[0,:,:,:1]
        temp[:, 128*2:,:1] = decoded_imgs[0,:,:,:1]
        temp[:,:,1]=temp[:,:,0]
        temp[:,:,2]=temp[:,:,0]
        temp = temp*255
        scipy.misc.imsave(CheckDir + str(epoch) + ".jpg", temp)

Epoch is: 1

Number of batches: 153

epoch_num: 1 batch_num: 0 loss: 0.089861

epoch_num: 1 batch_num: 1 loss: 0.059608

epoch_num: 1 batch_num: 2 loss: 0.050801

epoch_num: 1 batch_num: 3 loss: 0.061453

epoch_num: 1 batch_num: 4 loss: 0.053809

epoch_num: 1 batch_num: 5 loss: 0.041582

epoch_num: 1 batch_num: 6 loss: 0.030566

epoch_num: 1 batch_num: 7 loss: 0.027469

epoch_num: 1 batch_num: 8 loss: 0.031740

epoch_num: 1 batch_num: 9 loss: 0.024777

epoch_num: 1 batch_num: 10 loss: 0.019068

epoch_num: 1 batch_num: 11 loss: 0.014329

epoch_num: 1 batch_num: 12 loss: 0.016312

epoch_num: 1 batch_num: 13 loss: 0.012143

epoch_num: 1 batch_num: 14 loss: 0.014262

epoch_num: 1 batch_num: 15 loss: 0.013137

epoch_num: 1 batch_num: 16 loss: 0.015569

epoch_num: 1 batch_num: 17 loss: 0.015542

epoch_num: 1 batch_num: 18 loss: 0.012439

epoch_num: 1 batch_num: 19 loss: 0.010900

epoch_num: 1 batch_num: 20 loss: 0.011487

epoch_num: 1 batch_num: 21 loss: 0.010835

epoch_num: 1 batch_num: 22 

epoch_num: 2 batch_num: 36 loss: 0.008381

epoch_num: 2 batch_num: 37 loss: 0.005971

epoch_num: 2 batch_num: 38 loss: 0.006577

epoch_num: 2 batch_num: 39 loss: 0.007148

epoch_num: 2 batch_num: 40 loss: 0.007920

epoch_num: 2 batch_num: 41 loss: 0.006431

epoch_num: 2 batch_num: 42 loss: 0.007180

epoch_num: 2 batch_num: 43 loss: 0.006079

epoch_num: 2 batch_num: 44 loss: 0.007183

epoch_num: 2 batch_num: 45 loss: 0.008049

epoch_num: 2 batch_num: 46 loss: 0.007865

epoch_num: 2 batch_num: 47 loss: 0.007306

epoch_num: 2 batch_num: 48 loss: 0.007119

epoch_num: 2 batch_num: 49 loss: 0.006635

epoch_num: 2 batch_num: 50 loss: 0.007859

epoch_num: 2 batch_num: 51 loss: 0.006728

epoch_num: 2 batch_num: 52 loss: 0.007672

epoch_num: 2 batch_num: 53 loss: 0.006569

epoch_num: 2 batch_num: 54 loss: 0.006949

epoch_num: 2 batch_num: 55 loss: 0.006974

epoch_num: 2 batch_num: 56 loss: 0.007886

epoch_num: 2 batch_num: 57 loss: 0.007938

epoch_num: 2 batch_num: 58 loss: 0.007184

epoch_num: 

epoch_num: 3 batch_num: 72 loss: 0.007599

epoch_num: 3 batch_num: 73 loss: 0.005868

epoch_num: 3 batch_num: 74 loss: 0.006871

epoch_num: 3 batch_num: 75 loss: 0.006412

epoch_num: 3 batch_num: 76 loss: 0.006055

epoch_num: 3 batch_num: 77 loss: 0.006318

epoch_num: 3 batch_num: 78 loss: 0.006563

epoch_num: 3 batch_num: 79 loss: 0.005975

epoch_num: 3 batch_num: 80 loss: 0.005752

epoch_num: 3 batch_num: 81 loss: 0.006255

epoch_num: 3 batch_num: 82 loss: 0.005820

epoch_num: 3 batch_num: 83 loss: 0.006169

epoch_num: 3 batch_num: 84 loss: 0.005688

epoch_num: 3 batch_num: 85 loss: 0.005387

epoch_num: 3 batch_num: 86 loss: 0.005203

epoch_num: 3 batch_num: 87 loss: 0.005230

epoch_num: 3 batch_num: 88 loss: 0.005056

epoch_num: 3 batch_num: 89 loss: 0.005862

epoch_num: 3 batch_num: 90 loss: 0.005521

epoch_num: 3 batch_num: 91 loss: 0.006067

epoch_num: 3 batch_num: 92 loss: 0.007160

epoch_num: 3 batch_num: 93 loss: 0.006560

epoch_num: 3 batch_num: 94 loss: 0.006525

epoch_num: 

epoch_num: 4 batch_num: 108 loss: 0.006276

epoch_num: 4 batch_num: 109 loss: 0.006062

epoch_num: 4 batch_num: 110 loss: 0.004840

epoch_num: 4 batch_num: 111 loss: 0.005946

epoch_num: 4 batch_num: 112 loss: 0.005813

epoch_num: 4 batch_num: 113 loss: 0.004888

epoch_num: 4 batch_num: 114 loss: 0.005104

epoch_num: 4 batch_num: 115 loss: 0.005634

epoch_num: 4 batch_num: 116 loss: 0.006801

epoch_num: 4 batch_num: 117 loss: 0.005144

epoch_num: 4 batch_num: 118 loss: 0.005474

epoch_num: 4 batch_num: 119 loss: 0.005569

epoch_num: 4 batch_num: 120 loss: 0.004914

epoch_num: 4 batch_num: 121 loss: 0.005386

epoch_num: 4 batch_num: 122 loss: 0.004948

epoch_num: 4 batch_num: 123 loss: 0.005260

epoch_num: 4 batch_num: 124 loss: 0.005333

epoch_num: 4 batch_num: 125 loss: 0.006507

epoch_num: 4 batch_num: 126 loss: 0.005704

epoch_num: 4 batch_num: 127 loss: 0.006940

epoch_num: 4 batch_num: 128 loss: 0.005976

epoch_num: 4 batch_num: 129 loss: 0.005621

epoch_num: 4 batch_num: 130 loss

epoch_num: 5 batch_num: 143 loss: 0.004548

epoch_num: 5 batch_num: 144 loss: 0.004425

epoch_num: 5 batch_num: 145 loss: 0.005180

epoch_num: 5 batch_num: 146 loss: 0.003887

epoch_num: 5 batch_num: 147 loss: 0.004535

epoch_num: 5 batch_num: 148 loss: 0.005697

epoch_num: 5 batch_num: 149 loss: 0.004816

epoch_num: 5 batch_num: 150 loss: 0.005857

epoch_num: 5 batch_num: 151 loss: 0.004359

epoch_num: 5 batch_num: 152 loss: 0.004674

Epoch is: 6

Number of batches: 153

epoch_num: 6 batch_num: 0 loss: 0.004500

epoch_num: 6 batch_num: 1 loss: 0.005259

epoch_num: 6 batch_num: 2 loss: 0.004641

epoch_num: 6 batch_num: 3 loss: 0.004657

epoch_num: 6 batch_num: 4 loss: 0.004490

epoch_num: 6 batch_num: 5 loss: 0.004350

epoch_num: 6 batch_num: 6 loss: 0.004819

epoch_num: 6 batch_num: 7 loss: 0.004517

epoch_num: 6 batch_num: 8 loss: 0.005275

epoch_num: 6 batch_num: 9 loss: 0.005251

epoch_num: 6 batch_num: 10 loss: 0.003825

epoch_num: 6 batch_num: 11 loss: 0.005130

epoch_num: 6 batc

...

epoch_num: 221 batch_num: 93 loss: 0.002592

epoch_num: 221 batch_num: 94 loss: 0.002412

epoch_num: 221 batch_num: 95 loss: 0.002068

epoch_num: 221 batch_num: 96 loss: 0.002649

epoch_num: 221 batch_num: 97 loss: 0.002245

epoch_num: 221 batch_num: 98 loss: 0.002505

epoch_num: 221 batch_num: 99 loss: 0.002292

epoch_num: 221 batch_num: 100 loss: 0.002279

epoch_num: 221 batch_num: 101 loss: 0.002619

epoch_num: 221 batch_num: 102 loss: 0.002407

epoch_num: 221 batch_num: 103 loss: 0.002379

epoch_num: 221 batch_num: 104 loss: 0.002345

epoch_num: 221 batch_num: 105 loss: 0.002362

epoch_num: 221 batch_num: 106 loss: 0.002275

epoch_num: 221 batch_num: 107 loss: 0.002130

epoch_num: 221 batch_num: 108 loss: 0.002361

epoch_num: 221 batch_num: 109 loss: 0.002309

epoch_num: 221 batch_num: 110 loss: 0.002381

epoch_num: 221 batch_num: 111 loss: 0.002482

epoch_num: 221 batch_num: 112 loss: 0.002163

epoch_num: 221 batch_num: 113 loss: 0.002806

epoch_num: 221 batch_num: 114 loss: 0.002

epoch_num: 222 batch_num: 120 loss: 0.002594

epoch_num: 222 batch_num: 121 loss: 0.002563

epoch_num: 222 batch_num: 122 loss: 0.002414

epoch_num: 222 batch_num: 123 loss: 0.001991

epoch_num: 222 batch_num: 124 loss: 0.002562

epoch_num: 222 batch_num: 125 loss: 0.002547

epoch_num: 222 batch_num: 126 loss: 0.002413

epoch_num: 222 batch_num: 127 loss: 0.002535

epoch_num: 222 batch_num: 128 loss: 0.002309

epoch_num: 222 batch_num: 129 loss: 0.002203

epoch_num: 222 batch_num: 130 loss: 0.002013

epoch_num: 222 batch_num: 131 loss: 0.002369

epoch_num: 222 batch_num: 132 loss: 0.002279

epoch_num: 222 batch_num: 133 loss: 0.002883

epoch_num: 222 batch_num: 134 loss: 0.002093

epoch_num: 222 batch_num: 135 loss: 0.002560

epoch_num: 222 batch_num: 136 loss: 0.002382

epoch_num: 222 batch_num: 137 loss: 0.002680

epoch_num: 222 batch_num: 138 loss: 0.002077

epoch_num: 222 batch_num: 139 loss: 0.002494

epoch_num: 222 batch_num: 140 loss: 0.002228

epoch_num: 222 batch_num: 141 loss

epoch_num: 223 batch_num: 147 loss: 0.002492

epoch_num: 223 batch_num: 148 loss: 0.002311

epoch_num: 223 batch_num: 149 loss: 0.002097

epoch_num: 223 batch_num: 150 loss: 0.002760

epoch_num: 223 batch_num: 151 loss: 0.002131

epoch_num: 223 batch_num: 152 loss: 0.002411

Epoch is: 224

Number of batches: 153

epoch_num: 224 batch_num: 0 loss: 0.002382

epoch_num: 224 batch_num: 1 loss: 0.002255

epoch_num: 224 batch_num: 2 loss: 0.002286

epoch_num: 224 batch_num: 3 loss: 0.002284

epoch_num: 224 batch_num: 4 loss: 0.002508

epoch_num: 224 batch_num: 5 loss: 0.002963

epoch_num: 224 batch_num: 6 loss: 0.002729

epoch_num: 224 batch_num: 7 loss: 0.002311

epoch_num: 224 batch_num: 8 loss: 0.002267

epoch_num: 224 batch_num: 9 loss: 0.002537

epoch_num: 224 batch_num: 10 loss: 0.002179

epoch_num: 224 batch_num: 11 loss: 0.002668

epoch_num: 224 batch_num: 12 loss: 0.003093

epoch_num: 224 batch_num: 13 loss: 0.002344

epoch_num: 224 batch_num: 14 loss: 0.002382

epoch_num: 224 batch

epoch_num: 225 batch_num: 21 loss: 0.002317

epoch_num: 225 batch_num: 22 loss: 0.002814

epoch_num: 225 batch_num: 23 loss: 0.002539

epoch_num: 225 batch_num: 24 loss: 0.002425

epoch_num: 225 batch_num: 25 loss: 0.002363

epoch_num: 225 batch_num: 26 loss: 0.002366

epoch_num: 225 batch_num: 27 loss: 0.002200

epoch_num: 225 batch_num: 28 loss: 0.002424

epoch_num: 225 batch_num: 29 loss: 0.002578

epoch_num: 225 batch_num: 30 loss: 0.002307

epoch_num: 225 batch_num: 31 loss: 0.002418

epoch_num: 225 batch_num: 32 loss: 0.002062

epoch_num: 225 batch_num: 33 loss: 0.001954

epoch_num: 225 batch_num: 34 loss: 0.002608

epoch_num: 225 batch_num: 35 loss: 0.002981

epoch_num: 225 batch_num: 36 loss: 0.002222

epoch_num: 225 batch_num: 37 loss: 0.002555

epoch_num: 225 batch_num: 38 loss: 0.002936

epoch_num: 225 batch_num: 39 loss: 0.002091

epoch_num: 225 batch_num: 40 loss: 0.002271

epoch_num: 225 batch_num: 41 loss: 0.002351

epoch_num: 225 batch_num: 42 loss: 0.002174

epoch_num:

epoch_num: 226 batch_num: 49 loss: 0.002389

epoch_num: 226 batch_num: 50 loss: 0.002825

epoch_num: 226 batch_num: 51 loss: 0.002466

epoch_num: 226 batch_num: 52 loss: 0.002223

epoch_num: 226 batch_num: 53 loss: 0.002473

epoch_num: 226 batch_num: 54 loss: 0.002287

epoch_num: 226 batch_num: 55 loss: 0.001878

epoch_num: 226 batch_num: 56 loss: 0.002210

epoch_num: 226 batch_num: 57 loss: 0.002291

epoch_num: 226 batch_num: 58 loss: 0.002322

epoch_num: 226 batch_num: 59 loss: 0.002062

epoch_num: 226 batch_num: 60 loss: 0.002130

epoch_num: 226 batch_num: 61 loss: 0.002505

epoch_num: 226 batch_num: 62 loss: 0.002433

epoch_num: 226 batch_num: 63 loss: 0.002291

epoch_num: 226 batch_num: 64 loss: 0.002671

epoch_num: 226 batch_num: 65 loss: 0.002383

epoch_num: 226 batch_num: 66 loss: 0.002993

epoch_num: 226 batch_num: 67 loss: 0.002809

epoch_num: 226 batch_num: 68 loss: 0.002381

epoch_num: 226 batch_num: 69 loss: 0.002502

epoch_num: 226 batch_num: 70 loss: 0.002386

epoch_num: